In [1]:
import os, sys
import re
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Embedding,TimeDistributed
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import string
from pickle import dump
from unicodedata import normalize
from numpy import array
from pickle import load
from pickle import dump
from numpy.random import shuffle
import tensorflow as tf

C:\Users\PC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\PC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\PC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\PC\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1)

In [27]:
BATCH_SIZE = 64
EPOCHS = 30
LSTM_NODES =256
NUM_SENTENCES = 30000
MAX_SENTENCE_LENGTH = 50
MAX_NUM_WORDS = 20000
EMBEDDING_SIZE = 100

In [3]:
path='C:\\Users\PC\Desktop\data_set/deu.txt'
input_sentences = []
output_sentences = []
output_sentences_inputs = []

count = 0
for line in open(path, encoding="utf-8"):
    count += 1

    if count > NUM_SENTENCES:
        break

    if '\t' not in line:
        continue

    input_sentence, output,s = line.rstrip().split('\t')

    output_sentence = output + ' <eos>'
    output_sentence_input = '<sos> ' + output

    input_sentences.append(input_sentence)
    output_sentences.append(output_sentence)
    output_sentences_inputs.append(output_sentence_input)

print("num samples input:", len(input_sentences))
print("num samples output:", len(output_sentences))
print("num samples output input:", len(output_sentences_inputs))

num samples input: 30000
num samples output: 30000
num samples output input: 30000


In [4]:
print(input_sentences[172])
print(output_sentences[172])
print(output_sentences_inputs[172])

Go away.
Gehen Sie weg. <eos>
<sos> Gehen Sie weg.


In [5]:
input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(input_sentences)
input_integer_seq = input_tokenizer.texts_to_sequences(input_sentences)

word2idx_inputs = input_tokenizer.word_index
print('Total unique words in the input: %s' % len(word2idx_inputs))

max_input_len = max(len(sen) for sen in input_integer_seq)
print("Length of longest sentence in input: %g" % max_input_len)

Total unique words in the input: 4711
Length of longest sentence in input: 6


In [6]:
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index
print('Total unique words in the output: %s' % len(word2idx_outputs))

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)
print("Length of longest sentence in the output: %g" % max_out_len)

Total unique words in the output: 10919
Length of longest sentence in the output: 11


In [7]:
encoder_input_sequences = pad_sequences(input_integer_seq, maxlen=max_input_len)
print("encoder_input_sequences.shape:", encoder_input_sequences.shape)
print("encoder_input_sequences[172]:", encoder_input_sequences[172])

encoder_input_sequences.shape: (30000, 6)
encoder_input_sequences[172]: [ 0  0  0  0 23 96]


In [8]:
print(word2idx_inputs["i'm"])
print(word2idx_inputs["read"])

7
150


In [9]:
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_input_sequences.shape)
print("decoder_input_sequences[172]:", decoder_input_sequences[172])

decoder_input_sequences.shape: (30000, 11)
decoder_input_sequences[172]: [  2 156   6 147   0   0   0   0   0   0   0]


In [10]:
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')
print("decoder_input_sequences.shape:", decoder_output_sequences.shape)
print("decoder_input_sequences[172]:", decoder_output_sequences[172])

decoder_input_sequences.shape: (30000, 11)
decoder_input_sequences[172]: [156   6 147   1   0   0   0   0   0   0   0]


In [11]:
path_embeds='D:\\glove.6B/glove.6B.100d.txt'

In [12]:

glove_file = open(path_embeds, encoding="utf8")
embeddings_dictionary=dict()
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

In [13]:
num_words = min(MAX_NUM_WORDS, len(word2idx_inputs) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_SIZE))
for word, index in word2idx_inputs.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [14]:
print(embeddings_dictionary["read"])

[-3.6109e-01  3.9902e-01 -1.3540e-01 -2.6579e-01  1.2291e-01  1.2668e+00
 -3.1164e-01 -2.6861e-01 -4.7820e-02 -3.2642e-01  2.9340e-01  8.5681e-02
 -2.5868e-01  5.8440e-04  2.0940e-01  1.2185e-01  6.3982e-01  2.3537e-01
 -5.4758e-01 -2.2416e-01  3.9379e-02 -4.5530e-01 -3.2071e-02 -3.5783e-01
  3.6941e-01 -3.7714e-01 -3.9291e-01 -1.7099e-01  5.0699e-01  4.2622e-01
  3.1585e-01  4.1306e-01 -7.2180e-02  1.4811e-01 -2.1061e-01  5.5002e-01
 -1.6419e-01  3.9613e-02  2.2926e-01 -1.8447e-01 -7.0578e-01  1.7350e-01
 -4.5185e-01 -5.3340e-01 -7.4506e-01 -2.0174e-01 -1.4373e-01 -6.0950e-01
 -2.2733e-01 -8.0096e-01  9.4654e-01  2.3967e-01  5.8265e-01  5.3151e-01
 -3.8951e-01 -1.5985e+00 -3.5368e-01  4.6296e-01  1.3289e+00  5.9022e-03
 -1.1194e-04  1.1528e+00 -6.6395e-01 -1.0022e+00  4.2442e-01  6.9708e-03
  7.7096e-01  1.0685e+00 -2.9703e-01  1.9858e-01 -4.6281e-02  6.5891e-01
  7.8765e-01 -7.1153e-01  1.7818e-02  8.0047e-01  1.5151e-01  1.1654e-01
 -3.1582e-01 -6.6401e-01  1.7498e-01 -5.8295e-01 -7

In [15]:
embedding_matrix[143]

array([ 7.67670013e-03,  2.57719994e-01,  2.93870002e-01, -4.18220013e-01,
        4.64750007e-02,  3.24360013e-01,  4.34249997e-01,  5.83119988e-01,
       -8.87579992e-02,  8.93549994e-02, -3.36170010e-02,  5.92159986e-01,
        4.99049991e-01,  5.36769986e-01,  2.82660007e-01,  1.45310000e-01,
        6.90370023e-01, -3.31779987e-01, -6.68099999e-01,  4.12250012e-01,
        2.15289995e-01,  2.97659993e-01,  5.56259990e-01,  1.11950003e-01,
       -2.21609999e-03, -1.61520004e-01,  4.24199998e-01, -4.70910013e-01,
       -4.03829992e-01,  3.15369993e-01,  7.49130011e-01,  7.35879987e-02,
       -2.30289996e-01,  2.79529989e-01,  7.21069992e-01, -1.75830007e-01,
        8.85310024e-02,  3.35319996e-01,  2.63390005e-01,  2.56030001e-02,
       -8.22130024e-01, -3.81139994e-01,  1.16300002e-01, -3.24680001e-01,
        1.77310005e-01,  3.31319988e-01,  2.36330003e-01,  1.08939998e-01,
       -2.33480006e-01, -3.63359988e-01,  6.76199973e-01, -7.20240027e-02,
        8.88689980e-02,  

In [16]:
decoder_targets_one_hot = np.zeros((
        len(input_sentences),
        max_out_len,
        num_words_output
    ),
    dtype='float32'
)
decoder_targets_one_hot.shape


(30000, 11, 10920)

In [17]:
for i, d in enumerate(decoder_output_sequences):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

In [18]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [19]:
from tensorflow.keras.regularizers import l2

embedding_layer = Embedding(num_words, EMBEDDING_SIZE, weights=[embedding_matrix], input_length=max_input_len,trainable=False)
intial_lstm=LSTM(LSTM_NODES, return_sequences=True)
encoder = LSTM(LSTM_NODES, return_state=True)
decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [20]:
from tensorflow.keras.layers import Dropout
with tf.device('/device:GPU:0'):
    encoder_inputs_placeholder = Input(shape=(max_input_len,))
    X=embedding_layer(encoder_inputs_placeholder)
    X1 = Dropout(0.2)(X)
    X_encoder, h, c=encoder(X1)
    encoder_states = [h, c]
    decoder_inputs_placeholder = Input(shape=(max_out_len,))
    decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)
    decoder_dense = Dense(num_words_output, activation='softmax')
    X11 = Dropout(0.3)(decoder_outputs)
    decoder_outputs = decoder_dense(X11)

In [25]:
from tensorflow.keras.optimizers import Adam

model = Model([encoder_inputs_placeholder,
  decoder_inputs_placeholder], decoder_outputs)
model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 6)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 6, 100)       471200      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 11)]         0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 6, 100)       0           embedding[0][0]                  
____________________________________________________________________________________________

In [28]:
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.1, shuffle=True
)

Train on 27000 samples, validate on 3000 samples
Epoch 1/30
27000/27000 [==============================] - 106s 4ms/sample - loss: 0.8276 - acc: 0.8625 - val_loss: 1.5393 - val_acc: 0.7782
Epoch 2/30
27000/27000 [==============================] - 85s 3ms/sample - loss: 0.7095 - acc: 0.8714 - val_loss: 1.5034 - val_acc: 0.7811
Epoch 3/30
27000/27000 [==============================] - 81s 3ms/sample - loss: 0.6088 - acc: 0.8799 - val_loss: 1.4720 - val_acc: 0.7843
Epoch 4/30
27000/27000 [==============================] - 80s 3ms/sample - loss: 0.5254 - acc: 0.8876 - val_loss: 1.4726 - val_acc: 0.7852
Epoch 5/30
27000/27000 [==============================] - 75s 3ms/sample - loss: 0.4590 - acc: 0.8944 - val_loss: 1.4603 - val_acc: 0.7879
Epoch 6/30
27000/27000 [==============================] - 77s 3ms/sample - loss: 0.4056 - acc: 0.9015 - val_loss: 1.4530 - val_acc: 0.7874
Epoch 7/30
27000/27000 [==============================] - 83s 3ms/sample - loss: 0.3628 - acc: 0.9087 - val_loss: 1.

In [31]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)
decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)
decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)
decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [32]:
idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [33]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)


In [51]:
i = np.random.choice(len(input_sentences))
input_seq = encoder_input_sequences[i:i+1]
translation = translate_sentence(input_seq)
print('-')
print('Input:', input_sentences[i])
print('Response:', translation[:])

-
Input: It's bitter cold.
Response: es ist bitterkalt.


In [47]:
encoder_model.save('encoding_model')

In [48]:
decoder_model.save('decoding_model')